<a href="https://colab.research.google.com/github/serikovalera/ML_leti_projects/blob/main/%D1%82%D0%B5%D1%81%D1%821.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Лабораторная №1

In [ ]:
#%pip install transformers tokenizers datasets evaluate accelerate
#%pip install --upgrade git+https://github.com/dask/s3fs

####Подготовка датасета
Соберем ссылки на романы Анна Каренина, Война и Мир и др. в один список:

In [ ]:
urls = [
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0039.shtml",
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0040.shtml",
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0050.shtml",
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0060.shtml",
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0070.shtml",
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0080.shtml",
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0090.shtml",
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_1860_dekabristy.shtml",
]

In [ ]:
import html
import re

import requests


def download(url):
    return requests.get(url).text


# code borrowed from here: https://github.com/pallets/markupsafe/blob/0.23/markupsafe/__init__.py#L21
striptags_re = re.compile(r"(<!--.*?-->|<[^>]*>)")
entity_re = re.compile(r"&([^;]+);")


def to_text(s):
    return html.unescape(striptags_re.sub("", s))


def beautify(s):
    lines = [x.strip() for x in s.split("\n") if x.strip() != ""]
    for i in range(min(100, len(lines))):
        if lines[i] == "-->":
            break
    return "\n".join(lines[i + 1 :] if i < 100 else lines)


with open("dataset.txt", "w", encoding="utf-8") as f:
    for u in urls:
        text = beautify(to_text(download(u)))
        f.write(text + "\n\n")




Для обучения своего токенизатора используем библиотеку tokenizers:

In [ ]:
import tokenizers as tok
import transformers as tr

tokenizer = tok.Tokenizer(tok.models.BPE(unk_token="[UNK]"))
tokenizer.pre_tokenizer = tok.pre_tokenizers.Whitespace()
trainer = tok.trainers.BpeTrainer(special_tokens=["[PAD]"])
tokenizer.train(["dataset.txt"], trainer)
tokenizer.enable_padding()

In [ ]:
tokenizer.encode("Иван Сигизмундович подошел к окну и закашлялся. Вечерело.").tokens

['Иван',
 'С',
 'иг',
 'изму',
 'н',
 'до',
 'вич',
 'подошел',
 'к',
 'окну',
 'и',
 'закашлялся',
 '.',
 'Вечер',
 'ело',
 '.']

In [ ]:
vocab = tokenizer.get_vocab()
ttokenizer = tr.PreTrainedTokenizerFast(tokenizer_object=tokenizer)
len(vocab)

30000

In [ ]:
config = tr.GPT2Config(
    vocab_size=len(vocab),
    bos_token_id=tokenizer.token_to_id("[CLS]"),
    eos_token_id=tokenizer.token_to_id("[EOS]"),
)
gpt = tr.GPT2LMHeadModel(config)

In [ ]:
res = gpt.generate(
    **ttokenizer("Мне нравится ", return_tensors="pt"),
    max_new_tokens=50,
    top_k=3,
    do_sample=True
)
ttokenizer.decode(res[0])

'Мне нравится боку замолчал замолчал замолчал бессмысленные дные придви придви дорогое Тимох поглядела безупре хуже любов боку боку кончилось эгоизма барыни кончилось кончилось кончилось знанию знанию землю землю Пло Пло Пло ими ими кончилось кончилось кончилось кончилось эгоизма форм покупки дядюшки дядюшки дядюшки дядюшки дядюшки хуже las старшая водами столба столба столба'

In [ ]:
import datasets

dataset = datasets.load_dataset("text", data_files="dataset.txt")
dataset["train"][13]

Generating train split: 0 examples [00:00, ? examples/s]

{'text': 'Он взял своею большою рукой меня за руку, и пожал так крепко, честно, только что не больно. Я думала, что он поцелует мою руку, и нагнулась было к нему, но он еще раз пожал мне руку и прямо в глаза посмотрел своим твердым и веселым взглядом.'}

In [ ]:
def tokenize(x):
    x = ttokenizer(x["text"])
    x["labels"] = x["input_ids"].copy()
    return x


ds = dataset.map(tokenize, batched=True, remove_columns=["text"])
ds["train"][0]

Map:   0%|          | 0/60782 [00:00<?, ? examples/s]

{'input_ids': [9585, 9736, 3192],
 'token_type_ids': [0, 0, 0],
 'attention_mask': [1, 1, 1],
 'labels': [9585, 9736, 3192]}

In [ ]:
from itertools import chain

block_size = 1024

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

dsb = ds.map(group_texts, batched=True)

Map:   0%|          | 0/60782 [00:00<?, ? examples/s]

In [ ]:
targs = tr.TrainingArguments(
    output_dir="gpt2-scratch",
    num_train_epochs=30,
    learning_rate=5e-5,
    warmup_steps=200,
    save_steps=1500,
)
trainer = tr.Trainer(
    gpt,
    args=targs,
    train_dataset=dsb["train"],
    tokenizer=ttokenizer,
    data_collator=tr.default_data_collator,  # tr.DataCollatorForLanguageModeling(tokenizer=ttokenizer,mlm=False)
)

<ipython-input-16-34047ea34211>:8: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = tr.Trainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

In [ ]:
#ttokenizer = tr.PreTrainedTokenizerFast(tokenizer_object=tokenizer)
res = gpt.generate(
    **ttokenizer("Пьер закашлялся и", return_tensors="pt").to("cuda"),
    max_new_tokens=150,
    do_sample=True
)
ttokenizer.decode(res[0])